In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.callbacks import EarlyStopping


# Constants
IMG_SIZE = (100, 100)
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 1e-4
MARGIN = 0.5
THRESHOLD = 0.5

# Paths
POSITIVE_ANCHOR_DIR = r'C:\Users\moham\Documents\Facial_Recognition\data\positive'
LFW_NEGATIVE_DIR = r'C:\Users\moham\Documents\Facial_Recognition\data\negative'

# Load and preprocess images
def load_images_from_directory(directory):
    images = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, filename)
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
                img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
                images.append(img)
    return np.array(images)

positive_images = load_images_from_directory(POSITIVE_ANCHOR_DIR)
negative_images = load_images_from_directory(LFW_NEGATIVE_DIR)

# Triplet Creation
def create_triplets(positive_images, negative_images):
    """Create triplet data for training."""
    anchors, positives, negatives = [], [], []
    for anchor in positive_images:
        positive = positive_images[np.random.choice(len(positive_images))]
        negative = negative_images[np.random.choice(len(negative_images))]
        anchors.append(anchor)
        positives.append(positive)
        negatives.append(negative)
    return np.array(anchors), np.array(positives), np.array(negatives)

# Generate Triplets
anchors, positives, negatives = create_triplets(positive_images, negative_images)

# Train-Test Split
train_size = int(0.8 * len(anchors))
train_anchors, test_anchors = anchors[:train_size], anchors[train_size:]
train_positives, test_positives = positives[:train_size], positives[train_size:]
train_negatives, test_negatives = negatives[:train_size], negatives[train_size:]


# Embedding Model
def create_embedding_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
    x = Flatten()(base_model.output)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    return Model(inputs=base_model.input, outputs=x, name="embedding")

embedding_model = create_embedding_model()

# Triplet Loss Function
def triplet_loss(y_true, y_pred):
    anchor, positive, negative = y_pred[:, 0], y_pred[:, 1], y_pred[:, 2]
    positive_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    negative_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    return tf.reduce_mean(tf.maximum(positive_dist - negative_dist + MARGIN, 0))

# Siamese Model
anchor_input = Input(name="anchor", shape=(100, 100, 3))
positive_input = Input(name="positive", shape=(100, 100, 3))
negative_input = Input(name="negative", shape=(100, 100, 3))

anchor_embedding = embedding_model(anchor_input)
positive_embedding = embedding_model(positive_input)
negative_embedding = embedding_model(negative_input)

output = tf.stack([anchor_embedding, positive_embedding, negative_embedding], axis=1)

siamese_model = Model(inputs=[anchor_input, positive_input, negative_input], outputs=output, name="SiameseNetwork")
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=triplet_loss)

# EarlyStopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,          # Stop after 5 epochs with no improvement
    restore_best_weights=True  # Restore the weights of the best epoch
)

# Train the Model
history = siamese_model.fit(
    [train_anchors, train_positives, train_negatives],  # Inputs
    np.zeros(len(train_anchors)),                      # Dummy labels for triplet loss
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=([test_anchors, test_positives, test_negatives], np.zeros(len(test_anchors))),
    callbacks=[early_stopping]  # Add callbacks here
)

embedding_model.save("final_embedding_model.h5")
print("Final embedding model saved as final_embedding_model.h5")

# Evaluate the Model
def evaluate_model(embedding_model, test_anchors, test_positives, test_negatives, threshold=THRESHOLD):
    """Evaluate the model with accuracy, precision, and recall."""
    anchor_embeds = embedding_model.predict(test_anchors)
    positive_embeds = embedding_model.predict(test_positives)
    negative_embeds = embedding_model.predict(test_negatives)

    # Compute distances
    positive_distances = np.linalg.norm(anchor_embeds - positive_embeds, axis=1)
    negative_distances = np.linalg.norm(anchor_embeds - negative_embeds, axis=1)

    # Predictions based on threshold
    predictions = positive_distances < negative_distances

    # Ground truth: positive pairs are 1, negative pairs are 0
    ground_truth = np.ones(len(predictions))

    # Evaluate metrics
    accuracy = accuracy_score(ground_truth, predictions)
    precision = precision_score(ground_truth, predictions, zero_division=0)
    recall = recall_score(ground_truth, predictions, zero_division=0)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

for threshold in np.arange(0.1, 1.0, 0.1):
    print(f"Evaluating with threshold {threshold:.1f}")
    evaluate_model(embedding_model, test_anchors, test_positives, test_negatives, threshold=threshold)



Epoch 1/100
18/18 [==============================] - 11s 190ms/step - loss: 27.1537 - val_loss: 2.8949
Epoch 2/100
18/18 [==============================] - 2s 105ms/step - loss: 0.6558 - val_loss: 2.9316
Epoch 3/100
18/18 [==============================] - 2s 108ms/step - loss: 0.5067 - val_loss: 2.6727
Epoch 4/100
18/18 [==============================] - 2s 104ms/step - loss: 0.3402 - val_loss: 2.6822
Epoch 5/100
18/18 [==============================] - 2s 108ms/step - loss: 0.0000e+00 - val_loss: 2.3435
Epoch 6/100
18/18 [==============================] - 2s 108ms/step - loss: 0.0000e+00 - val_loss: 1.9286
Epoch 7/100
18/18 [==============================] - 2s 105ms/step - loss: 0.3727 - val_loss: 2.1924
Epoch 8/100
18/18 [==============================] - 2s 105ms/step - loss: 0.0000e+00 - val_loss: 2.5676
Epoch 9/100
18/18 [==============================] - 2s 106ms/step - loss: 0.0000e+00 - val_loss: 2.8533
Epoch 10/100
18/18 [==============================] - 2s 105ms/step - los

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score

embedding_model = load_model("final_embedding_model.h5")

# Paths for test data
TEST_ANCHORS_DIR = r"C:\Users\moham\Documents\Facial_Recognition\data\test_anchors"
TEST_POSITIVES_DIR = r"C:\Users\moham\Documents\Facial_Recognition\data\test_positives"
TEST_NEGATIVES_DIR = r"C:\Users\moham\Documents\Facial_Recognition\data\test_negatives"

# Constants
IMG_SIZE = (100, 100)


In [5]:
# Function to load and preprocess images
def load_images_from_directory(directory):
    images = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, filename)
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
                img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
                images.append(img)
    return np.array(images)

# Load test images
test_anchors = load_images_from_directory(TEST_ANCHORS_DIR)
test_positives = load_images_from_directory(TEST_POSITIVES_DIR)
test_negatives = load_images_from_directory(TEST_NEGATIVES_DIR)

In [7]:
def compute_distances(embedding_model, anchors, positives, negatives):
    # Debug: Print shapes
    print(f"Anchor input shape: {anchors.shape}")
    print(f"Positive input shape: {positives.shape}")
    print(f"Negative input shape: {negatives.shape}")

    # Ensure correct shapes
    anchors = anchors.reshape((-1, 100, 100, 3))
    positives = positives.reshape((-1, 100, 100, 3))
    negatives = negatives.reshape((-1, 100, 100, 3))

    # Debug: Check for empty arrays
    if len(anchors) == 0 or len(positives) == 0 or len(negatives) == 0:
        raise ValueError("One of the datasets (anchors, positives, negatives) is empty.")

    # Test single image prediction
    print("Testing single image prediction...")
    test_image = anchors[0:1]
    print(f"Test image shape: {test_image.shape}")
    embedding = embedding_model.predict(test_image)
    print(f"Embedding shape: {embedding.shape}")

    # Generate embeddings
    print("Generating embeddings...")
    anchor_embeds = embedding_model.predict(anchors)
    positive_embeds = embedding_model.predict(positives)
    negative_embeds = embedding_model.predict(negatives)

    # Compute distances
    positive_distances = np.linalg.norm(anchor_embeds - positive_embeds, axis=1)
    negative_distances = np.linalg.norm(anchor_embeds - negative_embeds, axis=1)

    return positive_distances, negative_distances


In [9]:
# Visualize distance distributions
def visualize_distance_distributions(positive_distances, negative_distances, initial_threshold=0.5):
    plt.hist(positive_distances, bins=30, alpha=0.7, label="Positive Distances", color="blue")
    plt.hist(negative_distances, bins=30, alpha=0.7, label="Negative Distances", color="red")
    plt.axvline(x=initial_threshold, color="green", linestyle="--", label=f"Initial Threshold ({initial_threshold})")
    plt.xlabel("Distance")
    plt.ylabel("Frequency")
    plt.legend()
    plt.title("Distance Distributions for Positive and Negative Pairs")
    plt.show()

positive_distances, negative_distances = compute_distances(
    embedding_model, test_anchors, test_positives, test_negatives
)

# Visualize distributions
visualize_distance_distributions(positive_distances, negative_distances)

Anchor input shape: (0,)
Positive input shape: (0,)
Negative input shape: (0,)


ValueError: One of the datasets (anchors, positives, negatives) is empty.

In [ ]:


# Load the model
# Reload model 
model = tf.keras.models.load_model('siamese_model.h5', 
                                   custom_objects={'L1Dist': L1Dist, 'BinaryCrossentropy': tf.losses.BinaryCrossentropy})

model.summary()

In [64]:
import cv2

def verify(model, detection_threshold=0.85, verification_threshold=0.7):
    """
    Verify the input image against the validation images in the verification folder.
    Preprocessing steps are integrated directly into the function.
    """
    # Results array
    results = []
    
    # Path to the input image
    input_img_path = os.path.join('application_data', 'input_image', 'input_image.jpg')
    
    # Load and preprocess the input image
    input_img = cv2.imread(input_img_path)
    input_img = cv2.resize(input_img, IMG_SIZE)  # Resize to match model input size
    input_img = input_img / 255.0  # Normalize to range [0, 1]
    input_img = np.expand_dims(input_img, axis=0)  # Add batch dimension
    
    # Iterate through all images in the verification_images folder
    for image_name in os.listdir(os.path.join('application_data', 'verification_images')):
        # Path to the validation image
        validation_img_path = os.path.join('application_data', 'verification_images', image_name)
        
        # Load and preprocess the validation image
        validation_img = cv2.imread(validation_img_path)
        validation_img = cv2.resize(validation_img, IMG_SIZE)  # Resize to match model input size
        validation_img = validation_img / 255.0  # Normalize to range [0, 1]
        validation_img = np.expand_dims(validation_img, axis=0)  # Add batch dimension
        
        # Make predictions
        result = model.predict([input_img, validation_img])
        results.append(result)
    
    # Convert results to a numpy array
    results = np.array(results)
    
    # Detection threshold: count positive predictions
    detection = np.sum(results > detection_threshold)
    
    # Verification threshold: ratio of positive predictions to total predictions
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images')))
    verified = verification > verification_threshold
    
    return results, verified


In [66]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(model, 0.9, 0.7)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step
True
1/1 [==============================] - 0s 18ms/step
False
1/1 [==============================] - 0s 20ms/step
True
1/1 [==============================] - 0s 18ms/step
False
1/1 [==============================] - 0s 20ms/step
True
1/1 [==============================] - 0s 19ms/step
True
1/1 [==============================] - 0s 19ms/step
False
1/1 [==============================] - 0s 18ms/step
False


In [24]:
import os
import shutil

# Define source and destination paths
user1_folder = r"C:\Users\moham\Documents\Facial_Recognition\data\user1"
verification_images_folder = r"C:\Users\moham\Documents\Facial_Recognition\application_data\verification_images"

# Check if both folders exist
if not os.path.exists(user1_folder):
    print(f"Source folder does not exist: {user1_folder}")
elif not os.path.exists(verification_images_folder):
    print(f"Destination folder does not exist: {verification_images_folder}")
else:
    # Get all files in the user1 folder
    user1_images = [f for f in os.listdir(user1_folder) if os.path.isfile(os.path.join(user1_folder, f))]

    # Ensure there are files to copy
    if not user1_images:
        print("No images found in the user1 folder.")
    else:
        # Copy up to 50 images
        files_to_copy = user1_images[:50]  # Get the first 50 files
        for file in files_to_copy:
            source_path = os.path.join(user1_folder, file)
            dest_path = os.path.join(verification_images_folder, file)
            
            # Copy the file
            shutil.copy(source_path, dest_path)
            print(f"Copied: {file}")
        
        print(f"Copied {len(files_to_copy)} files from {user1_folder} to {verification_images_folder}")


Copied: 04b0ea26-9ef9-11ef-8193-244bfedec36b.jpg
Copied: 04b61ac2-9ef9-11ef-b8e0-244bfedec36b.jpg
Copied: 04badeb9-9ef9-11ef-a6d8-244bfedec36b.jpg
Copied: 04bf549f-9ef9-11ef-a51a-244bfedec36b.jpg
Copied: 04cb3c80-9ef9-11ef-a3d2-244bfedec36b.jpg
Copied: 04cfe7a8-9ef9-11ef-b2ed-244bfedec36b.jpg
Copied: 04d989fa-9ef9-11ef-8b45-244bfedec36b.jpg
Copied: 04de6ffc-9ef9-11ef-9fb4-244bfedec36b.jpg
Copied: 04e352a9-9ef9-11ef-a7af-244bfedec36b.jpg
Copied: 04e834ee-9ef9-11ef-a113-244bfedec36b.jpg
Copied: 04ed1733-9ef9-11ef-b169-244bfedec36b.jpg
Copied: 04f1f97e-9ef9-11ef-8a9a-244bfedec36b.jpg
Copied: 04f6dbe8-9ef9-11ef-89ec-244bfedec36b.jpg
Copied: 04fc067b-9ef9-11ef-8c1b-244bfedec36b.jpg
Copied: 05079f12-9ef9-11ef-9b5b-244bfedec36b.jpg
Copied: 05113ba7-9ef9-11ef-bc60-244bfedec36b.jpg
Copied: 0515f2e4-9ef9-11ef-a859-244bfedec36b.jpg
Copied: 051ab1e1-9ef9-11ef-a4cc-244bfedec36b.jpg
Copied: 051f7bc3-9ef9-11ef-ba73-244bfedec36b.jpg
Copied: 0524ac5a-9ef9-11ef-a39c-244bfedec36b.jpg
Copied: 052ff80d-9ef

In [57]:
import os

verification_images_path = r"C:\Users\moham\Documents\Facial_Recognition\application_data\verification_images"
print("Verification Images Path:", verification_images_path)

if os.path.exists(verification_images_path):
    print("Directory exists.")
    files = os.listdir(verification_images_path)
    print("Files:", files)
    if len(files) == 0:
        print("No files found in the directory.")
    else:
        print("Files found:", files)
else:
    print("Directory does not exist.")

Verification Images Path: C:\Users\moham\Documents\Facial_Recognition\application_data\verification_images
Directory exists.
Files: ['Person_Name_1.jpg', 'Person_Name_10.jpg', 'Person_Name_11.jpg', 'Person_Name_12.jpg', 'Person_Name_13.jpg', 'Person_Name_14.jpg', 'Person_Name_15.jpg', 'Person_Name_16.jpg', 'Person_Name_17.jpg', 'Person_Name_18.jpg', 'Person_Name_19.jpg', 'Person_Name_2.jpg', 'Person_Name_20.jpg', 'Person_Name_21.jpg', 'Person_Name_22.jpg', 'Person_Name_23.jpg', 'Person_Name_24.jpg', 'Person_Name_25.jpg', 'Person_Name_26.jpg', 'Person_Name_27.jpg', 'Person_Name_28.jpg', 'Person_Name_29.jpg', 'Person_Name_3.jpg', 'Person_Name_30.jpg', 'Person_Name_31.jpg', 'Person_Name_32.jpg', 'Person_Name_33.jpg', 'Person_Name_34.jpg', 'Person_Name_35.jpg', 'Person_Name_36.jpg', 'Person_Name_37.jpg', 'Person_Name_38.jpg', 'Person_Name_39.jpg', 'Person_Name_4.jpg', 'Person_Name_40.jpg', 'Person_Name_41.jpg', 'Person_Name_42.jpg', 'Person_Name_43.jpg', 'Person_Name_44.jpg', 'Person_Name_

In [48]:
import os
import cv2

# Paths
verification_images_path = os.path.join('application_data', 'verification_images')

# Create directory if it doesn't exist
os.makedirs(verification_images_path, exist_ok=True)

def capture_verification_images(person_name, num_images=50):
    """Captures images for a person and saves them in the verification_images folder."""
    # Step 1: Clear existing images in verification_images folder
    for file in os.listdir(verification_images_path):
        file_path = os.path.join(verification_images_path, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
    print("Cleared all images in the verification_images folder.")

    # Step 2: Open the camera for image capture
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not access the camera.")
        return

    print(f"Capturing {num_images} images for {person_name}. Press 'v' to capture an image or 'q' to quit.")
    image_count = 0

    while cap.isOpened() and image_count < num_images:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to grab frame from the camera.")
            break

        # Crop and display the frame
        frame = frame[120:120+250, 200:200+250, :]  # Adjust cropping as needed
        cv2.imshow('Capture Verification Images', frame)

        # Save image when 'v' is pressed
        if cv2.waitKey(10) & 0xFF == ord('v'):
            image_filename = f"{person_name}_{image_count + 1}.jpg"
            image_path = os.path.join(verification_images_path, image_filename)
            cv2.imwrite(image_path, frame)
            print(f"Captured and saved image: {image_filename}")
            image_count += 1

        # Quit capturing if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            print("Quitting image capture.")
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Finished capturing {image_count} images for {person_name}.")


In [56]:
capture_verification_images("Person_Name", 50)


Cleared all images in the verification_images folder.
Capturing 50 images for Person_Name. Press 'v' to capture an image or 'q' to quit.
Captured and saved image: Person_Name_1.jpg
Captured and saved image: Person_Name_2.jpg
Captured and saved image: Person_Name_3.jpg
Captured and saved image: Person_Name_4.jpg
Captured and saved image: Person_Name_5.jpg
Captured and saved image: Person_Name_6.jpg
Captured and saved image: Person_Name_7.jpg
Captured and saved image: Person_Name_8.jpg
Captured and saved image: Person_Name_9.jpg
Captured and saved image: Person_Name_10.jpg
Captured and saved image: Person_Name_11.jpg
Captured and saved image: Person_Name_12.jpg
Captured and saved image: Person_Name_13.jpg
Captured and saved image: Person_Name_14.jpg
Captured and saved image: Person_Name_15.jpg
Captured and saved image: Person_Name_16.jpg
Captured and saved image: Person_Name_17.jpg
Captured and saved image: Person_Name_18.jpg
Captured and saved image: Person_Name_19.jpg
Captured and save

In [60]:
results

array([[[1.0000000e+00]],

       [[1.0000000e+00]],

       [[9.9999952e-01]],

       [[1.0000000e+00]],

       [[1.0000000e+00]],

       [[1.0000000e+00]],

       [[6.4580041e-01]],

       [[1.0000000e+00]],

       [[1.0000000e+00]],

       [[9.9999988e-01]],

       [[9.9982685e-01]],

       [[9.9995041e-01]],

       [[9.9823260e-01]],

       [[9.2989933e-01]],

       [[9.0175979e-07]],

       [[1.2797383e-09]],

       [[4.3770811e-07]],

       [[1.2313598e-04]],

       [[5.5669667e-04]],

       [[6.1525635e-02]],

       [[9.9754059e-01]],

       [[3.5003528e-02]],

       [[1.0000000e+00]],

       [[2.0158530e-05]],

       [[7.2927010e-04]],

       [[6.9818911e-06]],

       [[1.6047134e-03]],

       [[1.6636033e-01]],

       [[2.7247530e-01]],

       [[9.9171281e-01]],

       [[9.9999487e-01]],

       [[9.9871826e-01]],

       [[9.9719340e-01]],

       [[9.9999893e-01]],

       [[9.9999952e-01]],

       [[9.9971026e-01]],

       [[9.7354072e-01]],

 

In [68]:
np.sum(np.squeeze(results) > 0.01)

1

In [3]:


print("Positive directory:", POSITIVE_ANCHOR_DIR)
print("Negative directory:", LFW_NEGATIVE_DIR)


Positive directory: C:\Users\moham\Documents\Facial_Recognition\data\positive
Negative directory: C:\Users\moham\Documents\Facial_Recognition\data\negative
